# Task 3.2: Embedding and Extracting Hidden Messages

## Objective
The goal is to create a system that embeds a hidden message into an audio file using Least Significant Bit (LSB) steganography. The system should also be capable of extracting the hidden message.

## Implementation
The process includes:
1. Converting the message into binary form.
2. Embedding the binary message into randomly selected audio samples using LSB substitution.
3. Extracting the embedded message by reversing the embedding process.

In [1]:
# Import necessary libraries
import numpy as np  # For numerical computations
import wave  # For audio file handling
import random  # For generating random indices
import os  # For file operations

In [2]:
# Function to embed a message in an audio file
def embed_message(audio_filename, output_filename, message, seed=42, lsb_count=1):
    """
    Embed a message in an audio file using LSB steganography.

    Args:
        audio_filename (str): Path to the input audio file.
        output_filename (str): Path to the output audio file with the embedded message.
        message (str): Message to embed.
        seed (int): Seed for randomization.
        lsb_count (int): Number of least significant bits to use for embedding.
    """
    with wave.open(audio_filename, 'rb') as infile:
        params = infile.getparams()
        nframes = infile.getnframes()
        audio_data = infile.readframes(nframes)
        
        audio_signal = np.frombuffer(audio_data, dtype=np.int16).copy()
        message_binary = ''.join(format(ord(char), '08b') for char in message)
        
        if len(message_binary) > len(audio_signal) * lsb_count:
            raise ValueError("Audio file does not have enough capacity to embed the message.")
        
        random.seed(seed)
        indices = random.sample(range(len(audio_signal)), len(message_binary) // lsb_count)
        for i, bit_chunk in zip(indices, [message_binary[j:j+lsb_count] for j in range(0, len(message_binary), lsb_count)]):
            audio_signal[i] = (audio_signal[i] & ~(2**lsb_count - 1)) | int(bit_chunk, 2)
        
        with wave.open(output_filename, 'wb') as outfile:
            outfile.setparams(params)
            outfile.writeframes(audio_signal.tobytes())

In [3]:
# Function to extract a message from an audio file
def extract_message(audio_filename, message_length, seed=42, lsb_count=1):
    """
    Extract a hidden message from an audio file.

    Args:
        audio_filename (str): Path to the input audio file with the embedded message.
        message_length (int): Length of the message to extract (in characters).
        seed (int): Seed for randomization.
        lsb_count (int): Number of least significant bits used for embedding.

    Returns:
        str: The extracted message.
    """
    with wave.open(audio_filename, 'rb') as infile:
        nframes = infile.getnframes()
        audio_data = infile.readframes(nframes)
        audio_signal = np.frombuffer(audio_data, dtype=np.int16)
        
        random.seed(seed)
        indices = random.sample(range(len(audio_signal)), message_length * 8 // lsb_count)
        message_binary = ''.join(format(audio_signal[i] & (2**lsb_count - 1), f"0{lsb_count}b") for i in indices)
        
        message = ''.join(chr(int(message_binary[i:i+8], 2)) for i in range(0, len(message_binary), 8))
        return message

In [4]:
# Example usage
audio_file = "Ex3_sound5.wav"
output_file = "embedded_sound.wav"
message = "An eye for an eye makes the whole world blind"

embed_message(audio_file, output_file, message)
print("Message embedded successfully.")

extracted_message = extract_message(output_file, len(message))
print("Extracted message:", extracted_message)

Message embedded successfully.
Extracted message: An eye for an eye makes the whole world blind
